<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.4: Sequential Logic
**Prev: [Control Flow](2.3_control_flow.ipynb)**<br>
**Next: [FIR Filter](2.5_exercise.ipynb)**

## Motivation
상태 없이는 의미 있는 디지털 논리를 작성할 수 없습니다. 상태 없이는 의미 있는 디지털 논리를 작성할 수 없습니다. 의미있는 디지털 논리를 쓸 수 없다....

알겠어? 중간 결과를 저장하지 않고는 아무데도 갈 수 없기 때문입니다.

자, 그 나쁜 농담은 제쳐두고, 이 모듈은 Chisel에서 일반적인 순차 패턴을 표현하는 방법을 설명할 것입니다. 모듈이 끝나면 Chisel에서 시프트 레지스터를 구현하고 테스트할 수 있어야 합니다.

이 섹션이 당신에게 극적으로 감동을 주지 않을 것이라는 점을 강조하는 것이 중요합니다. Chisel의 힘은 새로운 순차 논리 패턴이 아니라 설계의 매개변수화에 있습니다. 그 기능을 시연하기 전에 이러한 순차적 패턴이 무엇인지 배워야 합니다. 따라서 이 섹션에서는 Chisel이 Verilog가 할 수 있는 일을 거의 수행할 수 있음을 보여줍니다. Chisel 구문을 배우기만 하면 됩니다.

## Setup

In [5]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp/source/load-ivy.sc"

In [6]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Registers

Chisel의 기본 상태 저장 요소는 `Reg`로 표시된 레지스터입니다.
`Reg`는 clock의 상승 에지까지 출력 값을 유지하며, 이때 입력 값을 취합니다.
기본적으로 모든 Chisel `Module`에는 설계의 모든 레지스터에서 사용되는 암시적 clock가 있습니다.
이렇게 하면 코드 전체에서 항상 동일한 clock을 지정하지 않아도 됩니다.

<span style="color:blue">**Example: Using a Register**</span><br>
다음 코드 블록은 입력을 받아 여기에 1을 더하고 레지스터의 입력으로 연결하는 모듈을 구현합니다.
*참고: 암시적 clock은 multi-clock 설계에서 재정의될 수 있습니다. 예제는 부록을 참조하십시오.*

In [7]:
class RegisterModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = Reg(UInt(12.W))
  register := io.in + 1.U
  io.out := register
}

test(new RegisterModule) { c =>
  for (i <- 0 until 100) {
    c.io.in.poke(i.U)
    c.clock.step(1)
    c.io.out.expect((i + 1).U)
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test RegisterModule Success: 0 tests passed in 102 cycles in 0.014277 seconds 7144.12 Hz
SUCCESS!!


defined class RegisterModule

레지스터는 `Reg(tpe)`를 호출하여 생성됩니다. 여기서 `tpe`는 원하는 레지스터 유형을 인코딩하는 변수입니다.
이 예에서 `tpe`는 12비트 `UInt`입니다.

위의 테스터가 하는 일을 보십시오.
`poke()` 호출과 `expect` 호출 사이에 `step(1)` 호출이 있습니다.
이것은 테스트 하네스가 클럭을 한 번 틱하도록 지시하여 레지스터가 입력을 출력으로 전달하도록 합니다.

`step(n)`을 호출하면 clock이 `n`번 tick할 것입니다.

예리한 관찰자는 조합 논리를 테스트하는 이전 테스터가 `step()`을 호출하지 않았음을 알 수 있습니다. 이는 입력에서 `poke()`를 호출하면 조합 논리를 통해 업데이트된 값이 즉시 전파되기 때문입니다. `step()`을 호출하는 것은 순차 논리에서 상태 요소를 업데이트하는 데만 필요합니다.

아래 코드 블록은 `RegisterModule`에 의해 생성된 Verilog를 보여줍니다.

Note:
* 모듈에는 추가하지 않은 클럭(및 재설정)에 대한 입력이 있습니다. 이것은 암시적 클럭입니다.
* 변수 `register`는 예상대로 `reg [11:0]`으로 표시됩니다.
* 시뮬레이션이 시작되기 전에 레지스터를 임의의 변수로 초기화한 `ifdef Randomize`에 의해 구획된 블록이 있습니다.
* `register`는 `posedge clock`에서 업데이트됩니다.

In [8]:
println(getVerilog(new RegisterModule))

Elaborating design...
Done elaborating.
module RegisterModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] register; // @[cmd6.sc 7:21]
  assign io_out = register; // @[cmd6.sc 9:10]
  always @(posedge clock) begin
    register <= io_in + 12'h1; // @[cmd6.sc 8:21]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DEL

한 가지 중요한 점은 Chisel이 유형(예: `UInt`)과 하드웨어 노드(예: 리터럴 `2.U` 또는 `myReg`의 출력)를 구별한다는 것입니다.
```scala
val myReg = Reg(UInt(2.W))
```
다음은 Reg가 모델로 데이터 유형을 필요로 하기 때문에 합법적입니다.
```scala
val myReg = Reg(2.U)
```
`2.U`는 이미 하드웨어 노드이고 모델로 사용할 수 없기 때문에 오류입니다.

<span style="color:blue">**Example: RegNext**</span><br>
Chisel에는 간단한 입력 연결이 있는 레지스터를 위한 편리한 레지스터 객체가 있습니다. 이전 `Module`은 다음 `Module`으로 단축될 수 있습니다. 이번에는 레지스터 비트폭을 지정할 필요가 없었습니다. 레지스터의 출력 연결(이 경우 `io.out`)에서 추론됩니다.

In [9]:
class RegNextModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  // register bitwidth is inferred from io.out
  io.out := RegNext(io.in + 1.U)
}

test(new RegNextModule) { c =>
  for (i <- 0 until 100) {
    c.io.in.poke(i.U)
    c.clock.step(1)
    c.io.out.expect((i + 1).U)
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test RegNextModule Success: 0 tests passed in 102 cycles in 0.010975 seconds 9293.85 Hz
SUCCESS!!


defined class RegNextModule

Verilog는 이전과 거의 같아 보이지만 명시적으로 정의되는 대신 레지스터 이름이 생성됩니다.

In [10]:
println(getVerilog(new RegNextModule))

Elaborating design...
Done elaborating.
module RegNextModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] REG; // @[cmd8.sc 8:20]
  assign io_out = REG; // @[cmd8.sc 8:10]
  always @(posedge clock) begin
    REG <= io_in + 12'h1; // @[cmd8.sc 8:27]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
   

---
# `RegInit`

`RegisterModule`의 레지스터는 시뮬레이션을 위해 임의의 데이터로 초기화되었습니다.
달리 지정되지 않는 한 레지스터에는 reset 값(또는 reset)이 없습니다.
주어진 값으로 재설정되는 레지스터를 만드는 방법은 `RegInit`를 사용하는 것입니다.

예를 들어, 0으로 초기화된 12비트 레지스터는 다음과 같이 생성할 수 있습니다.
아래 두 버전 모두 유효하며 동일한 작업을 수행합니다.

```scala
val myReg = RegInit(UInt(12.W), 0.U)
val myReg = RegInit(0.U(12.W))
```

첫 번째 버전에는 두 개의 인수가 있습니다.
첫 번째 인수는 데이터 유형과 너비를 지정한 유형 노드입니다.
두 번째 인수는 재설정 값(이 경우 0)을 지정한 하드웨어 노드입니다.

두 번째 버전에는 하나의 인수가 있습니다.
reset 값을 지정하는 하드웨어 노드이지만 일반적으로 '0.U'입니다.

<span style="color:blue">**Example: Initialized Register** </span><br>
다음은 0으로 초기화된 `RegInit()`을 사용하는 방법을 보여줍니다.

In [11]:
class RegInitModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = RegInit(0.U(12.W))
  register := io.in + 1.U
  io.out := register
}

println(getVerilog(new RegInitModule))

Elaborating design...
Done elaborating.
module RegInitModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] register; // @[cmd10.sc 7:25]
  wire [11:0] _T_1 = io_in + 12'h1; // @[cmd10.sc 8:21]
  assign io_out = register; // @[cmd10.sc 9:10]
  always @(posedge clock) begin
    if (reset) begin // @[cmd10.sc 7:25]
      register <= 12'h0; // @[cmd10.sc 7:25]
    end else begin
      register <= _T_1; // @[cmd10.sc 8:12]
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`en

defined class RegInitModule

생성된 Verilog에는 레지스터를 0으로 reset하기 위해 `if (reset)`를 확인하는 블록이 있습니다.
또한 이것은 `always @(posedge clock)` 블록 안에 있다는 점에 유의하십시오.
Chisel의 암시적 reset은 active high 및 동기식입니다.
레지스터는 reset이 호출되기 전에 여전히 임의의 정크로 초기화됩니다.
`PeekPokeTesters`는 테스트를 실행하기 전에 항상 reset을 호출하지만 `reset(n)` 함수를 사용하여 수동으로 reset을 호출할 수도 있습니다. 여기서 reset은 `n` 주기 동안 high값을 갖습니다.

---
# Control Flow
레지스터는 제어 흐름 측면에서 와이어와 매우 유사합니다.
그것들은 마지막 연결 의미 체계를 가지며 `when`, `elsewhen` 및 `otherwise`를 사용하여 조건부로 할당할 수 있습니다.

<span style="color:blue">**Example: Register Control Flow**</span><br>
다음 예에서는 조건부 레지스터 할당을 사용하여 입력 시퀀스에서 최대값을 찾습니다.

In [12]:
class FindMax extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(10.W))
    val max = Output(UInt(10.W))
  })

  val max = RegInit(0.U(10.W))
  when (io.in > max) {
    max := io.in
  }
  io.max := max
}

test(new FindMax) { c =>
    c.io.max.expect(0.U)
    c.io.in.poke(1.U)
    c.clock.step(1)
    c.io.max.expect(1.U)
    c.io.in.poke(3.U)
    c.clock.step(1)
    c.io.max.expect(3.U)
    c.io.in.poke(2.U)
    c.clock.step(1)
    c.io.max.expect(3.U)
    c.io.in.poke(24.U)
    c.clock.step(1)
    c.io.max.expect(24.U)
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test FindMax Success: 0 tests passed in 6 cycles in 0.001468 seconds 4088.45 Hz
SUCCESS!!


defined class FindMax

---
# Other Register Examples

레지스터에서 호출된 작업은 레지스터의 **출력**에서 수행되며 작업의 종류는 레지스터의 유형에 따라 다릅니다.

```scala
val reg: UInt = Reg(UInt(4.W))
```
즉, 값 `reg`가 `UInt` 유형이고 `+`, `-` 등과 같이 `UInt`로 일반적으로 수행할 수 있는 작업을 수행할 수 있음을 의미합니다.

레지스터와 함께 `UInt`를 사용하는 것으로 제한되지 않으며 기본 유형 `chisel3.Data`의 모든 하위 클래스를 사용할 수 있습니다. 여기에는 부호 있는 정수 및 기타 많은 항목에 대한 `SInt`가 포함됩니다.

<span style="color:blue">**Example: Comb Filter**</span><br>
다음 예는 Comb Filter를 보여줍니다.

In [13]:
class Comb extends Module {
  val io = IO(new Bundle {
    val in  = Input(SInt(12.W))
    val out = Output(SInt(12.W))
  })

  val delay: SInt = Reg(SInt(12.W))
  delay := io.in
  io.out := io.in - delay
}
println(getVerilog(new Comb))

Elaborating design...
Done elaborating.
module Comb(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] delay; // @[cmd12.sc 7:24]
  assign io_out = $signed(io_in) - $signed(delay); // @[cmd12.sc 9:19]
  always @(posedge clock) begin
    delay <= io_in; // @[cmd12.sc 8:9]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DE

defined class Comb

---
# Exercises
<span style="color:red">**Exercise: Shift Register**</span><br>
새로 발견한 registering 지식이 주어지면 LFSR에 대한 시프트 레지스터를 구현하는 모듈을 빌드하십시오. 구체적으로:
- 각 요소의 너비는 1비트입니다.
- 4비트 출력 신호가 있습니다.
- 시프트 레지스터의 다음 값인 단일 입력 비트를 사용합니다.
- 시프트 레지스터의 병렬 출력을 출력합니다. 최상위 비트는 시프트 레지스터의 마지막 요소이고 최하위 비트는 시프트 레지스터의 첫 번째 요소입니다. '고양이'가 유용할 수 있습니다.
- **출력은 `b0001`에서 초기화됩니다.**
- 각 클록 주기를 이동합니다(활성화 신호 없음).
- **Chisel에서 참고, subword 할당은 불법입니다**; `out(0) := in`과 같은 것은 작동하지 않습니다.

<img src="images/shifter4.svg" alt="shift register figure" style="width: 450px" />


기본 모듈 스켈레톤, 테스트 벡터 및 드라이버 호출이 아래에 제공됩니다. 첫 번째 레지스터가 제공되었습니다.

class MyShiftRegister(val init: Int = 1) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Bool())
    val out = Output(UInt(4.W))
  })

  val state = RegInit(UInt(4.W), init.U)

  ???
}

test(new MyShiftRegister()) { c =>
  var state = c.init
  for (i <- 0 until 10) {
    // poke in LSB of i (i % 2)
    c.io.in.poke(((i % 2) != 0).B)
    // update expected state
    state = ((state * 2) + (i % 2)) & 0xf
    c.clock.step(1)
    c.io.out.expect(state.U)
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  state := nextState
  io.out := state
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Shift Register (Optional)**</span><br>
Write a shift register that is parameterized by its delay (`n`), its initial value (`init`), and also has an enable input signal (`en`).

In [ ]:
// n is the output width (number of delays - 1)
// init state to init
class MyOptionalShiftRegister(val n: Int, val init: BigInt = 1) extends Module {
  val io = IO(new Bundle {
    val en  = Input(Bool())
    val in  = Input(Bool())
    val out = Output(UInt(n.W))
  })

  val state = RegInit(init.U(n.W))

  ???
}

// test different depths
for (i <- Seq(3, 4, 8, 24, 65)) {
  println(s"Testing n=$i")
  test(new MyOptionalShiftRegister(n = i)) { c =>
    val inSeq = Seq(0, 1, 1, 1, 0, 1, 1, 0, 0, 1)
    var state = c.init
    var i = 0
    c.io.en.poke(true.B)
    while (i < 10 * c.n) {
      // poke in repeated inSeq
      val toPoke = inSeq(i % inSeq.length)
      c.io.in.poke((toPoke != 0).B)
      // update expected state
      state = ((state * 2) + toPoke) & BigInt("1"*c.n, 2)
      c.clock.step(1)
      c.io.out.expect(state.U)

      i += 1
    }
  }
}
println("SUCCESS!!")

Testing n=3
Elaborating design...


<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  when (io.en) {
    state  := nextState
  }
  io.out := state
</pre></article></div></section></div>

---
# Appendix: Explicit clock and reset
Chisel 모듈에는 내부에 생성된 모든 레지스터에서 암시적으로 사용하는 기본 clock 및 reset이 있습니다.
이 기본 동작을 재정의할 수 있기를 원하는 경우가 있습니다. clock 또는 reset 신호를 생성하는 블랙박스가 있거나 다중 클럭 설계가 있을 수 있습니다.

Chisel은 이러한 경우를 처리하기 위한 구문을 제공합니다.
클록과 리셋은 별도로 또는 `withClock() {}`, `withReset() {}`, `withClockAndReset() {}`와 함께 재정의할 수 있습니다.
다음 코드 블록은 이러한 기능을 사용하는 예를 제공합니다.

한 가지 알아야 할 것은 `reset`(이 튜토리얼의 작성 시점)은 항상 동기식이며 `Bool` 유형이라는 것입니다.
Clock은 Chisel(`Clock`)에 고유한 유형을 가지며 그렇게 선언해야 합니다.
*`Bool`은 `asClock()`을 호출하여 `Clock`으로 변환할 수 있지만 어리석은 일을 하지 않도록 주의해야 합니다.*

또한 'Chisel 테스터'는 현재 다중 클록 설계를 완벽하게 지원하지 않습니다.

<span style="color:blue">**Example: Multi-Clock Module**</span><br>
다중 클럭 및 리셋 신호가 있는 모듈입니다.

In [15]:
// we need to import multi-clock features
import chisel3.experimental.{withClock, withReset, withClockAndReset}

class ClockExamples extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(10.W))
    val alternateReset    = Input(Bool())
    val alternateClock    = Input(Clock())
    val outImplicit       = Output(UInt())
    val outAlternateReset = Output(UInt())
    val outAlternateClock = Output(UInt())
    val outAlternateBoth  = Output(UInt())
  })

  val imp = RegInit(0.U(10.W))
  imp := io.in
  io.outImplicit := imp

  withReset(io.alternateReset) {
    // everything in this scope with have alternateReset as the reset
    val altRst = RegInit(0.U(10.W))
    altRst := io.in
    io.outAlternateReset := altRst
  }

  withClock(io.alternateClock) {
    val altClk = RegInit(0.U(10.W))
    altClk := io.in
    io.outAlternateClock := altClk
  }

  withClockAndReset(io.alternateClock, io.alternateReset) {
    val alt = RegInit(0.U(10.W))
    alt := io.in
    io.outAlternateBoth := alt
  }
}

println(getVerilog(new ClockExamples))

Elaborating design...
Done elaborating.
module ClockExamples(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  input        io_alternateReset,
  input        io_alternateClock,
  output [9:0] io_outImplicit,
  output [9:0] io_outAlternateReset,
  output [9:0] io_outAlternateClock,
  output [9:0] io_outAlternateBoth
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
  reg [31:0] _RAND_2;
  reg [31:0] _RAND_3;
`endif // RANDOMIZE_REG_INIT
  reg [9:0] imp; // @[cmd14.sc 14:20]
  reg [9:0] REG; // @[cmd14.sc 20:25]
  reg [9:0] REG_1; // @[cmd14.sc 26:25]
  reg [9:0] REG_2; // @[cmd14.sc 32:22]
  assign io_outImplicit = imp; // @[cmd14.sc 16:18]
  assign io_outAlternateReset = REG; // @[cmd14.sc 22:26]
  assign io_outAlternateClock = REG_1; // @[cmd14.sc 28:26]
  assign io_outAlternateBoth = REG_2; // @[cmd14.sc 34:25]
  always @(posedge clock) begin
    if (reset) begin // @[cmd14.sc 14:20]
      imp <= 10'h0; // @[cmd14.sc 14:20]
    end else begin

import chisel3.experimental.{withClock, withReset, withClockAndReset}


defined class ClockExamples

---
# Wrap Up

이 섹션을 완료했습니다. 이제 Chisel에서 레지스터를 만들고 순차 논리를 작성하는 방법을 배웠습니다. 즉, 실제 회로를 작성하기에 충분한 기본 빌딩 블록이 있음을 의미합니다.

다음 섹션에서는 우리가 배운 모든 것을 하나의 예제로 결합할 것입니다! 조금 더 격려가 필요하시면 Chisel 전문 사용자의 다음 말을 기억하십시오.

![BobRoss](http://i.qkme.me/3qbd5u.jpg)

---
# You're done!

[Return to the top.](#top)